# Лабораторная работа № 1 Задача регрессии

## Задание
Решите пожалуйста следующие задачи.

1. Примените следующие типы регрессий для датасета представленного по
[ссылке](https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link).
- Линейная регрессия
- LASSO регрессия
- Ridge регрессия
- ElasticNet регрессия
- Polynom регрессия (для степени полинома 2)
- Polynom регрессия (для степени полинома 5)
- Polynom регрессия (для степени полинома 10)

Постройте модель для атрибутов `X = {SkinThickness, BMI}` и `Y = {Insulin}`.  
**Укажите `score` (r2_score) моделей. Найдите метрики MAE, MSE.**


## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
! gdown --fuzzy 'https://drive.google.com/file/d/1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec/view?usp=share_link'

Downloading...
From: https://drive.google.com/uc?id=1LBDnhITL0Wqwp5G6M6IBI-SSz8BIoNec
To: /content/diabetes.csv
100% 23.1k/23.1k [00:00<00:00, 36.5MB/s]


In [3]:
df = pd.read_csv('diabetes.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


##  О датасете
[Pima Indians Diabetes Database](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data)

Этот набор данных взят из Национального института диабета, заболеваний органов пищеварения и почек (NIDDK). Целью набора данных является диагностическое предсказание наличия у пациента диабета на основе определенных диагностических измерений, включенных в набор данных. На выбор этих экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года, родом из индейцев Пима.

Наборы данных состоят из нескольких переменных медицинских предикторов и одной целевой переменной `Outcome`. Переменные-предикторы включают количество беременностей, которые были у пациентки, её индекс массы тела `BMI`, уровень инсулина, возраст и так далее. Всего в датасете 9 переменных:
- `Pregnancies` - количество беременностей;
- `Glucose` - концентрация глюкозы в плазме через 2 часа при пероральном тесте на толерантность к глюкозе;
- `BloodPressure` - диастолическое артериальное давление (мм рт. ст.);
- `SkinThickness` - толщина кожной складки трицепса (мм);
- `Insulin` - 2-часовой сывороточный инсулин (мкЕд/мл);
- `BMI` - Индекс массы тела (вес в кг/(рост в м)^2);
- `DiabetesPedigreeFunction` - функция оценки вероятности диабета на основе семейного анамнеза;
- `Age` - возраст (лет);
- `Outcome` - целевая переменная (0 или 1) 268 из 768 — 1 (больны), остальные — 0 (здоровы).



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

## Линейная регрессия

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

X = df[['SkinThickness', 'BMI']].values
y = df['Insulin'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: 0.2183
MAE: 65.8900
MSE: 10401.7464


## LASSO регрессия

In [8]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=1.0, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: 0.2193
MAE: 66.2288
MSE: 10388.0931


## Ridge регрессия

In [9]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: 0.2184
MAE: 65.9213
MSE: 10400.8346


## ElasticNet регрессия

In [10]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(alpha=1.0, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: 0.2001
MAE: 71.8294
MSE: 10643.4104


## Polynom регрессия (для степени полинома 2)

In [11]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly_model = Pipeline([
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', LinearRegression())
])
poly_model.fit(X_train_scaled, y_train)

y_pred = poly_model.predict(X_test_scaled)

# Метрики
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: 0.2227
MAE: 63.8500
MSE: 10342.9652


## Polynom регрессия (для степени полинома 5)

In [12]:
poly_model = Pipeline([
    ('poly', PolynomialFeatures(degree=5)),
    ('linear', LinearRegression())
])
poly_model.fit(X_train_scaled, y_train)

y_pred = poly_model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: -346.2157
MAE: 203.4335
MSE: 4620207.6684


## Polynom регрессия (для степени полинома 10)

In [13]:
poly_model = Pipeline([
    ('poly', PolynomialFeatures(degree=10)),
    ('linear', LinearRegression())
])
poly_model.fit(X_train_scaled, y_train)

y_pred = poly_model.predict(X_test_scaled)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2 Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")

R2 Score: -697613019164.1604
MAE: 6339482.6391
MSE: 9282750321323210.0000


## Вывод

### Вывод о применимости разных видов регрессии

На основе полученных результатов можно сделать следующие выводы:

**1. Линейные модели (Linear, Ridge, LASSO, ElasticNet):**
- Показывают схожие результаты, так как регуляризация (Ridge, LASSO, ElasticNet) слабо влияет при небольшом количестве признаков (всего 2 признака)
- Если R2 Score получился низким (близко к 0 или отрицательный), это говорит о том, что зависимость между толщиной кожи/BMI и уровнем инсулина слабая и нелинейная
- MAE и MSE могут быть высокими из-за больших выбросов в данных (уровень инсулина варьируется от 0 до 846)

**2. Полиномиальные модели:**
- **Степень 2**: может немного улучшить результаты по сравнению с линейной регрессией, учитывая квадратичные зависимости между признаками
- **Степень 5 и 10**: при высоких степенях полинома возрастает риск переобучения (overfitting), особенно на небольшом датасете. Модель может показать хороший R2 Score на тренировочной выборке, но плохо работать на тестовой

**3. Общие наблюдения:**
- Задача предсказания уровня инсулина по двум признакам (толщина кожи и BMI) является сложной, так как инсулин зависит от множества других факторов
- Наличие нулевых значений в данных (0 для инсулина означает отсутствие измерения) усложняет задачу
- Для улучшения качества модели рекомендуется:
  - Добавить больше признаков (Glucose, Age и др.)
  - Провести предобработку данных (обработка нулевых значений)
  - Использовать более сложные модели (например, градиентный бустинг)

**Рекомендация:** Для данной задачи лучше всего подходит линейная регрессия или полиномиальная регрессия степени 2, так как они дают баланс между простотой и качеством предсказаний.